# 编译与运行环境

C/C++是静态语言,需要编译后才能执行.而编译的工具就是编译器.最常见的C++编译器有:

+ `GNU G++`开源大佬,linux上编译器的的代名词,在osx上也可以使用homebrew安装,windows上则有MinGW作为其移植版.
+ `llvm+Clang`苹果资助的编译器,也是osx下默认的编译器.有不少黑科技,比如C++的repl,jit技术等,一点不像保守的苹果资助的东西.
+ `MSVC`微软家的御用编译器,vs内置的编译器.只能在windows下使用,要在命令行中使用需要将其加入环境变量`Path`


对于C/C++来说,编译其本质就是将源文件转化为二进制文件或者可执行文件的过程,我们以一个小程序来作为例子,使用`gnu g++`工具来演示整个编译过程:

In [3]:
%%writefile ../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.cpp
#include <stdio.h>

int main(){
    printf("hello world");
    return 0;
}

Writing ../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.cpp


## 步骤一:解读头文件,结合源文件,组合生成真正的待编译代码
    
这一步可以使用`-E`指令实现.这个过程只激活预处理,不生成文件,而是输出到标准输出.因此你需要把它重定向到一个输出文件里.,一般来说我们用后缀`.i`保存这个文件.

这一步实际上做这些事儿:

+ 宏的替换
+ 注释的消除
+ 还有找到相关的库文件
+ 执行模板,将模板翻译成源码的实现
+ 将#include文件的全部内容插入.若用`<>`括起文件则在系统的INCLUDE目录中寻找文件,若用`" "`括起文件则在当前目录中寻找文件.

In [6]:
!g++ -E ../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.cpp

# 1 "../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.cpp"
# 1 "<built-in>" 1
# 1 "<built-in>" 3
# 336 "<built-in>" 3
# 1 "<command line>" 1
# 1 "<built-in>" 2
# 1 "../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.cpp" 2
# 1 "/usr/include/stdio.h" 1 3 4
# 64 "/usr/include/stdio.h" 3 4
# 1 "/usr/include/sys/cdefs.h" 1 3 4
# 533 "/usr/include/sys/cdefs.h" 3 4
# 1 "/usr/include/sys/_symbol_aliasing.h" 1 3 4
# 534 "/usr/include/sys/cdefs.h" 2 3 4
# 599 "/usr/include/sys/cdefs.h" 3 4
# 1 "/usr/include/sys/_posix_availability.h" 1 3 4
# 600 "/usr/include/sys/cdefs.h" 2 3 4
# 65 "/usr/include/stdio.h" 2 3 4
# 1 "/usr/include/Availability.h" 1 3 4
# 172 "/usr/include/Availability.h" 3 4
# 1 "/usr/include/AvailabilityInternal.h" 1 3 4
# 173 "/usr/include/Availability.h" 2 3 4
# 66 "/usr/include/stdio.h" 2 3 4

# 1 "/usr/include/_types.h" 1 3 4
# 27 "/usr/include/_types.h" 3 4
# 1 "/usr/include/sys/_types.h" 1 3 4
# 33 "/usr/include/sys/_types.h" 3 4
# 1 "/usr/include/machine/_type

## 步骤二:将代码转译成汇编语言

这一步可以使用`-S`指令实现.这个过程就是将上一步的这串字符转译成汇编代码.汇编代码说白了就是二进制代码中操作与资源的的替代而已.因此说C/C++和汇编其实也就一步之遥.这也是为什么很多时候嵌入式设备使用C语言编写的原因.


生成`helloworld.s`文件,`.s`文件表示是汇编文件，用编辑器打开就都是汇编指令了


In [8]:
!g++ -o ../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.s \
-S ../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.cpp

In [9]:
!cat ../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.s

	.section	__TEXT,__text,regular,pure_instructions
	.macosx_version_min 10, 11
	.globl	_main
	.align	4, 0x90
_main:                                  ## @main
	.cfi_startproc
## BB#0:
	pushq	%rbp
Ltmp0:
	.cfi_def_cfa_offset 16
Ltmp1:
	.cfi_offset %rbp, -16
	movq	%rsp, %rbp
Ltmp2:
	.cfi_def_cfa_register %rbp
	subq	$16, %rsp
	leaq	L_.str(%rip), %rdi
	movl	$0, -4(%rbp)
	movb	$0, %al
	callq	_printf
	xorl	%ecx, %ecx
	movl	%eax, -8(%rbp)          ## 4-byte Spill
	movl	%ecx, %eax
	addq	$16, %rsp
	popq	%rbp
	retq
	.cfi_endproc

	.section	__TEXT,__cstring,cstring_literals
L_.str:                                 ## @.str
	.asciz	"hello world"


.subsections_via_symbols


## 步骤三:将汇编语言的代码再编译为二进制代码

这一步使用`-c`指令,将汇编代码编译位二进制代码.注意这依然不是可执行文件,因为其中使用的函数很可能并不包含在二进制文件中.

In [10]:
!g++ -o ../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.o -c ../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.cpp

In [11]:
!cat ../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.o

����                         �                          �              �                    __text          __TEXT                  *             �       �            __cstring       __TEXT          *              J                             __compact_unwind__LD            8               X     �                    __eh_frame      __TEXT          X       @       x               h            $       
            -                                       _main _printf  

## 步骤四: 链接相关的二进制代码生成可执行文件

这一步就是将与文件相关的函数库引入以生成最终的可执行文件.

In [15]:
!g++ -o ../codes/bin/Cpp_tools/Cpp_compiler/section1/helloworld \
../codes/src/Cpp_tools/Cpp_compiler/section1/helloworld.o

In [17]:
!../codes/bin/Cpp_tools/Cpp_compiler/section1/helloworld

hello world

# 多文件编译

在第一helloworld那章已经提过如何编译一个多文件构成的项目,这边以一个稍微复杂些的例子来演示

In [33]:
%%writefile ../codes/include/Cpp_tools/Cpp_compiler/section2/binary_vector.hpp
#ifndef BINARY_VECTOR_H
#define BINARY_VECTOR_H//一般是文件名的大写 头文件结尾写上一行
namespace Vector{
typedef struct {
    float x;
    float y;
} BINARY_VECTOR;
}

#endif

Overwriting ../codes/include/Cpp_tools/Cpp_compiler/section2/binary_vector.hpp


In [34]:
%%writefile ../codes/include/Cpp_tools/Cpp_compiler/section2/unary_operator.hpp
#ifndef UNARY_OPERATOR_H
#define UNARY_OPERATOR_H//一般是文件名的大写 头文件结尾写上一行
#include "binary_vector.hpp"
#include <math.h>
namespace Vector{
float mod(Vector::BINARY_VECTOR);
}
#endif

Overwriting ../codes/include/Cpp_tools/Cpp_compiler/section2/unary_operator.hpp


In [35]:
%%writefile ../codes/src/Cpp_tools/Cpp_compiler/section2/unary_operator.cpp
#include "unary_operator.hpp"
float Vector::mod(Vector::BINARY_VECTOR a){
    float result = sqrt(a.x*a.x+a.y*a.y);
    return result;
}

Overwriting ../codes/src/Cpp_tools/Cpp_compiler/section2/unary_operator.cpp


In [40]:
%%writefile ../codes/include/Cpp_tools/Cpp_compiler/section2/binary_operator.hpp
#ifndef BINARY_OPERATOR_H
#define BINARY_OPERATOR_H//一般是文件名的大写 头文件结尾写上一行
#include "binary_vector.hpp"
namespace Vector{
Vector::BINARY_VECTOR add(Vector::BINARY_VECTOR,Vector::BINARY_VECTOR);
float mul(Vector::BINARY_VECTOR,Vector::BINARY_VECTOR);
}
#endif

Writing ../codes/include/Cpp_tools/Cpp_compiler/section2/binary_operator.hpp


In [41]:
%%writefile ../codes/src/Cpp_tools/Cpp_compiler/section2/binary_operator.cpp
#include "binary_operator.hpp"
Vector::BINARY_VECTOR Vector::add(Vector::BINARY_VECTOR a,Vector::BINARY_VECTOR b){
    Vector::BINARY_VECTOR result = {a.x+b.x,a.y+b.y};
    return result;
}
float Vector::mul(Vector::BINARY_VECTOR a,Vector::BINARY_VECTOR b){
    float result = a.x*b.x+a.y*b.y;
    return result;
}

Overwriting ../codes/src/Cpp_tools/Cpp_compiler/section2/binary_operator.cpp


In [42]:
%%writefile ../codes/test/Cpp_tools/Cpp_compiler/section2/test.cpp
#include <stdio.h>
#include "binary_vector.hpp"
#include "binary_operator.hpp"
#include "unary_operator.hpp"

int main(void){
    Vector::BINARY_VECTOR A = {1,2},B = {3,4};
    printf("mod(A)=%f\n",Vector::mod(A));
    printf("mod(B)=%f\n",Vector::mod(B));
    printf("mul(A,B)=%f\n",Vector::mul(A,B));
    Vector::BINARY_VECTOR result = Vector::add(A,B);
    printf("add(A,B)=<%f,%f>\n",result.x,result.y);
}


Overwriting ../codes/test/Cpp_tools/Cpp_compiler/section2/test.cpp


In [43]:
!g++ -o ../codes/bin/Cpp_tools/Cpp_compiler/section2/test \
-I ../codes/include/Cpp_tools/Cpp_compiler/section2 \
../codes/src/Cpp_tools/Cpp_compiler/section2/*.cpp \
../codes/test/Cpp_tools/Cpp_compiler/section2/test.cpp

In [44]:
!../codes/bin/Cpp_tools/Cpp_compiler/section2/test

mod(A)=2.236068
mod(B)=5.000000
mul(A,B)=11.000000
add(A,B)=<4.000000,6.000000>


## 定义对外的接口

# 链接

## 静态连接库

## 动态连接库